In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import expm
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
%gui qt5
import matplotlib.pyplot as plt

from scipy.spatial import distance

### Plotting

In [323]:
#set up plottig GUI
app = QtGui.QApplication([])
pg.setConfigOption('background','w')  

In [325]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

### Helper Functions

In [2]:
def predictStep(x, A):
    xPred = np.matmul(A,x)
    return xPred

In [3]:
def innovationStep(xPred,z, H):
    innovation = np.subtract(z, np.matmul(H, xPred)) #innovation
    return innovation

In [4]:
def UpdateStep(xPred, nu, K):
    xNew = np.add(xPred, np.matmul(K, innovation))
    return xNew

In [5]:
def readMeasurements():
    #read in measurements from csv
    rawCentroidData = pd.read_csv('2PeopleCentroids.csv', header=None)
    #find headers and frames within headers
    #each header has the structure X, Y, CentroidNumber
    #below each header is the frame data
    centroidFramesCartesianMeasurement = list()
    headerFound = False
    for rowIndex in range(0, rawCentroidData.shape[0]):
        row = rawCentroidData.loc[rowIndex]
        if row[0] == 'X' and row[1] == 'Y' and row[2] == 'CentroidNumber':
            if headerFound: 
                #actual data was found last frame and this frame actual data is found again
                #past frame ended so add to list
                centroidFramesCartesianMeasurement.append(frame)
                frame = pd.DataFrame([])
            else:
                #header found
                headerFound = True
                frame = pd.DataFrame([])
            #data should be following
        elif headerFound:
            if np.isnan(np.float(row[0])) and np.isnan(np.float(row[1])):
                #empty row 
                centroidFramesCartesianMeasurement.append(frame)
                headerFound = False
                #only time its going to be NaN if the frame is completely empty
            else:
                #actual data
                X = np.float(row[0])
                Y = np.float(row[1])
                CentroidNumber = int(row[2])
                if len(frame) == 0:
                    frame = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
                else:
                    data = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
                    frame = pd.concat([frame,data])
    
    return centroidFramesCartesianMeasurement

In [6]:
#convert x,y to r,theta
def convertCartesianToPolar(cartesianMeasurementCentroidDf):
    measurementR = np.sqrt(cartesianMeasurementCentroidDf['X']**2 + cartesianMeasurementCentroidDf['Y']**2)
    measurementTheta = np.arctan2(cartesianMeasurementCentroidDf['Y'],cartesianMeasurementCentroidDf['X'])
    polarCentroidDf = pd.DataFrame({'CentroidNumber':np.arange(0,max(cartesianMeasurementCentroidDf['CentroidNumber'])+1), 
                                   'MeasuredRange': measurementR,
                                   'MeasuredTheta': measurementTheta})
    return polarCentroidDf

In [7]:
def createDistanceMatrix(rThetaMeasurement,previousFrame):
    #create distance matrix
    distanceMatrix = np.eye(previousFrame.shape[0], rThetaMeasurement.shape[0])
    for centroid in previousFrame['CentroidNumber']:
        prevCentroidInfo = previousFrame.loc[centroid]
        #find distances between a selected prev centroid and the new centroids
        distances = np.sqrt(np.repeat(np.square(prevCentroidInfo.FilteredRange), repeats=rThetaMeasurement.shape[0]) + \
        np.square(rThetaMeasurement['MeasuredRange']) - \
        2*np.repeat(prevCentroidInfo.FilteredRange, repeats=rThetaMeasurement.shape[0]) * \
        rThetaMeasurement['MeasuredRange'] * np.cos(rThetaMeasurement['MeasuredTheta']-prevCentroidInfo.FilteredTheta))
    #     assign distances
        distanceMatrix[centroid,:] = distances
    return distanceMatrix

In [8]:
def dataAssociation(rThetaMeasurement, previousFrame):
    #create distance matrix
    distanceMatrix = createDistanceMatrix(rThetaMeasurement,previousFrame)
    association = np.full((max(distanceMatrix.shape[0], distanceMatrix.shape[1]), 2), np.nan)

    #associate
    numberOfLoops = min(distanceMatrix.shape[0], distanceMatrix.shape[1])
    for loopIterator in range(0, numberOfLoops):
        previousCentroid, measuredCentroid = np.where(np.min(distanceMatrix) == distanceMatrix)
    #     rThetaMeasurement.at[measuredCentroid[0],'CentroidNumber'] = previousCentroid[0] #associate with old centroid 
        association[loopIterator,0]= previousCentroid[0] #fill association matrix
        association[loopIterator,1]= measuredCentroid[0] #fill association matrix
        distanceMatrix[previousCentroid[0], :] = np.Inf
        distanceMatrix[:,measuredCentroid[0]] = np.Inf

    if np.isnan(association).any(): #if any NaN's still in the association matrix - mismatch alert
        if distanceMatrix.shape[0] > distanceMatrix.shape[1]:
            #more predictions than observations
            unassociatedPredictions = [pred for pred in list(previousFrame['CentroidNumber']) if pred not in list(association[:,0])]
            association[np.isnan(association[:,0]),0] = unassociatedPredictions
        else:
            #more observations than predictions
            #find which observation has not been associated
            unassociatedObservations = [observation for observation in list(rThetaMeasurement['CentroidNumber']) if observation not in list(association[:,1])]
            association[np.isnan(association[:,1]),1] = unassociatedObservations
            
    associationDf = pd.DataFrame(association, columns=['Predicted', 'Measured'])
    
    return associationDf

### Main Code

In [9]:
#initialize variables
deltaT = 50*10**-3 #50ms

#system matrix
A = np.array([
    [1, deltaT, 0,0],
    [0,1,0,0],
    [0,0,1,deltaT],
    [0,0,0,1]
])
#state transition matrix
F = expm(A*deltaT)
#output matrix
H = np.array([[1,0,0,0],
              [0,0,1,0]])
#covariance matrices
Q = np.eye(4)
R = np.ones(2).reshape(-1,1)
Pc = np.eye(4)

Pd = np.add(np.matmul(np.matmul(A,Pc), np.transpose(A)), Q) #prediction covariance
S = np.add(R, np.matmul(np.matmul(H, Pd), np.transpose(H))) #innovation covariance
K = np.matmul(Pd, np.matmul(np.transpose(H), np.linalg.inv(S))) #kalman gain
Pupdate = np.subtract(Pd, np.matmul(np.matmul(K, S), np.transpose(K))) #update covariance

filteredFramesPolar = list()

In [24]:
#read in measurements
centroidFramesCartesianMeasurement= readMeasurements()
#filtered frames (polaar measurements)
filteredFramesPolar = list()

for centroidFrame in centroidFramesCartesianMeasurement:
    print(centroidFrame)
    filteredFramePolar = np.array([])
    
    #first frame that is valid
    if len(filteredFramesPolar) == 0 and len(centroidFrame) > 0: #kalman filter initialization
        rThetaMeasurement = convertCartesianToPolar(centroidFrame)
        for centroid in rThetaMeasurement['CentroidNumber']:
            x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
            
            #predict
            xPred = predictStep(x, A)
            #measurement
            centroidInformation = rThetaMeasurement.loc[centroid]
            measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),
                                         axis=1)
            innovation = innovationStep(xPred, measurement, H)
            #update
            xUpdate = UpdateStep(xPred, innovation, K)
            #add in centroid number
            xUpdate = np.expand_dims(np.append(xUpdate, centroid), axis=1)
            #add value to array
            if len(filteredFramePolar) == 0:
                filteredFramePolar = xUpdate
            else:
                filteredFramePolar = np.vstack(filteredFramePolar,xUpdate)
        
    #valid frames
    elif len(centroidFrame) >= 0 and len(filteredFramesPolar) != 0:
        if len(centroidFrame) != 0:
            rThetaMeasurement = convertCartesianToPolar(centroidFrame)
        else:
            rThetaMeasurement = pd.DataFrame([], columns=['CentroidNumber', 'MeasuredRange', 'MeasuredTheta'])
            
        #perform data association
        associationDf = dataAssociation(rThetaMeasurement, filteredFramesPolar[-1])
        previousFrame = filteredFramesPolar[-1]
        
        for centroidIndex in associationDf.index:

            #predict
            predictedCentroid = associationDf.loc[centroidIndex]['Predicted']
            if np.isnan(predictedCentroid):
                x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
            else:
                xPred = previousFrame.loc[previousFrame['CentroidNumber'] == predictedCentroid]
                x = np.expand_dims(xPred.values[0][:4], axis=1)

            xPred = predictStep(x, A)
            #measurement
            associatedMeasuredCentroid = associationDf.loc[centroidIndex]['Measured']
            if not(np.isnan(associatedMeasuredCentroid)):
                #if associated centroid exists
                centroidInformation = rThetaMeasurement.loc[associatedMeasuredCentroid]
                measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),axis=1)
                innovation = innovationStep(xPred, measurement, H)
                #update
                xUpdate = UpdateStep(xPred, innovation, K)
            else:
                xUpdate = xPred
            #add in centroid number
            xUpdate = np.append(xUpdate, predictedCentroid)
            #add value to array
            if len(filteredFramePolar) == 0:
                filteredFramePolar = xUpdate
            else:
                filteredFramePolar = np.hstack((filteredFramePolar,xUpdate))  
    
    else:
        continue
        
    filteredPolarDf = pd.DataFrame(np.transpose(filteredFramePolar))
    filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity', 'CentroidNumber']
    filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
    filteredFramesPolar.append(filteredPolarDf) 

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
          X         Y  CentroidNumber
0 -1.590903  3.145848               0
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFr

KeyError: 1

In [25]:
xUpdate

array([[3.06731904],
       [3.37528748],
       [0.07270694],
       [0.08039654],
       [3.06731904],
       [3.37528748],
       [0.0745558 ],
       [0.0822454 ],
       [2.37759704],
       [1.99075248],
       [0.05548542],
       [0.04582638],
       [2.37759704],
       [1.99075248],
       [0.05733428],
       [0.04767524],
       [0.        ]])

In [23]:
filteredPolarDf

,0
0,2.135049
1,0.053310
2,0.648813
3,0.016200
4,0.000000


In [327]:
#plot
xPositions = np.array([])
yPositions = np.array([])
for centroidFrame in filteredFramesPolar:
    x = np.multiply(centroidFrame.FilteredRange, np.cos(centroidFrame.FilteredTheta))
    y = np.multiply(centroidFrame.FilteredRange, np.sin(centroidFrame.FilteredTheta))
    if len(xPositions) == 0:
        xPositions = x
    else:
        xPositions = np.append(xPositions, x)       
    if len(yPositions) == 0:
        yPositions = y
    else:
        yPositions = np.append(yPositions,y)

s1.setData(xPositions, yPositions)
QtGui.QApplication.processEvents() 

In [328]:
xUpdate

array([[ 1.4701899 ],
       [-0.0329203 ],
       [ 2.05171494],
       [ 0.02318668]])

## Unit Tests

In [162]:
#test centroid dataset
#centroidDf_A contains 2 centroids
centroidDfCartesian_A = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([-1,0]), 
                             'Y':np.array([3,1])})
#centroidDf_B contains 2 centroids and continues on from centroidDf_A
centroidDfCartesian_B = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([0.5,-1]), 
                             'Y':np.array([0.7,2.5])})
#centroidDf_C contains 3 centroids continuing from centroidDf_B
centroidDfCartesian_C = pd.DataFrame({'CentroidNumber':np.arange(0,3), 
                             'X':np.array([-1,0.7, 0.3]), 
                             'Y':np.array([1.2,0.5, 3.2])})

centroidDfCartesian_D = pd.DataFrame([])

#contains the centroid frames, where each frame represents a mmWave capture frame
centroidFramesCartesianMeasurement = [centroidDfCartesian_A, centroidDfCartesian_B,centroidDfCartesian_D]

In [310]:
previousFrame = filteredFramesPolar[-1]
rThetaMeasurement = convertCartesianToPolar(centroidDfCartesian_C)
filteredFramePolar = np.array([])

distanceMatrix = createDistanceMatrix(rThetaMeasurement,previousFrame)
association = np.full((max(distanceMatrix.shape[0], distanceMatrix.shape[1]), 2), np.nan)

#associate
numberOfLoops = min(distanceMatrix.shape[0], distanceMatrix.shape[1])
for loopIterator in range(0, numberOfLoops):
    previousCentroid, measuredCentroid = np.where(np.min(distanceMatrix) == distanceMatrix)
#     rThetaMeasurement.at[measuredCentroid[0],'CentroidNumber'] = previousCentroid[0] #associate with old centroid 
    association[loopIterator,0]= previousCentroid[0] #fill association matrix
    association[loopIterator,1]= measuredCentroid[0] #fill association matrix
    distanceMatrix[previousCentroid[0], :] = np.Inf
    distanceMatrix[:,measuredCentroid[0]] = np.Inf

if np.isnan(association).any(): #if any NaN's still in the association matrix - mismatch alert
    if distanceMatrix.shape[0] > distanceMatrix.shape[1]:
        #more predictions than observations
        unassociatedPredictions = [pred for pred in list(previousFrame['CentroidNumber']) if pred not in list(association[:,0])]
        association[np.isnan(association[:,0]),0] = unassociatedPredictions
    else:
        #more observations than predictions
        #find which observation has not been associated
        unassociatedObservations = [observation for observation in list(rThetaMeasurement['CentroidNumber']) if observation not in list(association[:,1])]
        association[np.isnan(association[:,1]),1] = unassociatedObservations
        
associationDf = pd.DataFrame(association, columns=['Predicted', 'Measured'])

for centroidIndex in associationDf.index:
    
    #predict
    predictedCentroid = associationDf.loc[centroidIndex]['Predicted']
    if np.isnan(predictedCentroid):
        x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
    else:
        xPred = previousFrame.loc[previousFrame['CentroidNumber'] == predictedCentroid]
        x = np.expand_dims(xPred.values[0][:4], axis=1)
    
    xPred = predictStep(x, A)
    #measurement
    associatedMeasuredCentroid = associationDf.loc[centroidIndex]['Measured']
    if not(np.isnan(associatedMeasuredCentroid)):
        #if associated centroid exists
        centroidInformation = rThetaMeasurement.loc[associatedMeasuredCentroid]
        measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),axis=1)
        innovation = innovationStep(xPred, measurement, H)
        #update
        xUpdate = UpdateStep(xPred, innovation, K)
    else:
        xUpdate = xPred
    
    #add value to array
    if len(filteredFramePolar) == 0:
        filteredFramePolar = xUpdate
    else:
        filteredFramePolar = np.hstack((filteredFramePolar,xUpdate))
        
filteredPolarDf = pd.DataFrame(np.transpose(filteredFramePolar))
filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity']
filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
filteredFramesPolar.append(filteredPolarDf)    

In [311]:
filteredPolarDf

,FilteredRange,FilteredDoppler,FilteredTheta,FilteredAngularVelocity,CentroidNumber
0,1.277990,0.031801,1.038007,0.025929,0
1,0.605751,0.015125,1.309236,0.032690,1
2,2.041926,0.050984,0.305214,0.007621,2


In [301]:
centroidIndex

1

In [304]:
associationDf

,Predicted,Measured
0,0.0,1.0
1,NaN,0.0
2,NaN,2.0
